In [0]:
!git clone https://github.com/pdhruv93/Lane-Detection-In-Video.git

Cloning into 'Lane-Detection-In-Video'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [1]:
%cd Lane-Detection-In-Video
!ls

/content/Lane-Detection-In-Video
Lane_Detector.ipynb  test_image.jpg  test_video.mp4


In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [0]:
def convertToCannyImage(image):
    #convert to grayscale
    gray_image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    #remove noise from image
    blurred_image=cv2.GaussianBlur(gray_image,(5,5),0)

    #detect intensity gradient in image
    canny_image=cv2.Canny(blurred_image,50,150)
    
    return canny_image

In [0]:
def regionOfInterest(image):
    height=image.shape[0]
    
    #defining region of interest
    triangle=np.array([[(200,height),(1100,height),(550,250)]])
    #here we have created 2d array as fillPoly method expects array of polygon.currently we have only 1 ploygon(triangle) 
    
    #creating black image
    black_image=np.zeros_like(image)
    
    #creating mask-- 255 is white color lines
    mask=cv2.fillPoly(black_image,triangle,255)
    
    #applying mask on original image
    masked_image=cv2.bitwise_and(image,mask)
    
    return masked_image

In [0]:
#display lines over a image
def displayLines(image,lines):
    line_image=np.zeros_like(image) #black image
    if lines is not None:
        for line in lines:
            #print(line) --output like [[704 418 927 641]] this is 2d array representing [[x1,y1,x2,y2]] for each line
            x1,y1,x2,y2=line.reshape(4) #converting to 1d array []

            #draw line over black image --(255,0,0) tells we want to draw blue line (b,g,r) values
            #10 is line thickness
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

In [0]:
def getSmoothLines(image,lines):
    left_fit=[]  #will hold m,c parameters for left side lines
    right_fit=[] #will hold m,c parameters for right side lines
    
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        #polyfit gives slope(m) and intercept(c) values from input points
        #last parameter 1 is for linear..so it will give linear parameters m,c
        parameters=np.polyfit((x1,x2),(y1,y2),1)
        slope=parameters[0]
        intercept=parameters[1]
        
        if slope<0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
    
    #take averages of all intercepts and slopes separately and get 1 single value for slope,intercept
    #axis=0 means vertically...see its always (row,column)...so row is always 0 position.
    #so axis 0 means over row(vertically)
    left_fit_average=np.average(left_fit,axis=0)
    right_fit_average=np.average(right_fit,axis=0)
    
    #now we have got m,c parameters for left and right line, we need to know x1,y1 x2,y2 parameters
    left_line=getLineCoordinatesFromParameters(image,left_fit_average)
    right_line=getLineCoordinatesFromParameters(image,right_fit_average)
    return np.array([left_line,right_line])

In [0]:
def getLineCoordinatesFromParameters(image,line_parameters):
        slope=line_parameters[0]
        intercept=line_parameters[1]
        y1=image.shape[0] #since line willl always start from bottom of image
        y2=int(y1*(3/5)) #some random point at 3/5
        x1=int((y1-intercept)/slope)
        x2=int((y2-intercept)/slope)
        return np.array([x1,y1,x2,y2])

In [20]:
#video input
cap=cv2.VideoCapture("test_video.mp4")

writer = None #Initialize video writer

# try to determine the total number of frames in the video file
try:
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("[INFO] {} total frames in video".format(total))
except:
    # an error occurred while trying to determine the total number of frames in the video file
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

currentFrameNumber=1

try:
  while(cap.isOpened()):
    (grabbed, frame)=cap.read()
    
    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed:
      break
    #break if current frame count exceeds total count
    if(currentFrameNumber > total):
      break
    
    #break if q from keyboard is read
    if(cv2.waitKey(1) == ord('q')):
      break
     
    #print("Grabbed frame.....{}".format(currentFrameNumber))
    
    
    canny_image=convertToCannyImage(frame)
    
    masked_image=regionOfInterest(canny_image)
    lines=cv2.HoughLinesP(masked_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    
    lines_image=displayLines(frame,lines) #lines over black image
    
    combo_image=cv2.addWeighted(frame,0.8,lines_image,1,1)
    
    smooth_lines=getSmoothLines(frame,lines)
    smooth_lines_image=displayLines(frame,smooth_lines) #lines over black image
    smooth_lines_combo_image=cv2.addWeighted(frame,0.8,smooth_lines_image,1,1)
    
    # check if the video writer is None
    if writer is None:
      # initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter("lane_video.avi", fourcc, 30,(smooth_lines_combo_image.shape[1], smooth_lines_combo_image.shape[0]), True)
    
    # write the output frame to disk
    writer.write(smooth_lines_combo_image)
    
    currentFrameNumber=currentFrameNumber+1

except:
  #traceback.print_exc()
  pass
    
# release the file pointers
print("[INFO] cleaning up...")

cap.release()
cv2.destroyAllWindows()

[INFO] 1295 total frames in video
[INFO] cleaning up...


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [0]:
#code for saving the output video
from google.colab import files
files.download('lane_video.avi')